In [2]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
from urllib.request import urlopen, Request

# Script Scrape Data

In [19]:
# Định nghĩa hàm clean_text ngoài vòng lặp để tối ưu
def clean_text(text):
    return '\n'.join(filter(None, (line.strip() for line in text.split('\n'))))

In [20]:
# Hàm crawl dữ liệu cho từng URL
def crawl_data_for_each_normal_url(html, url, date_crawl):
    
    soup = BeautifulSoup(html, "html.parser")
    if not soup:
        failed_urls.append(url)
        
    # Job title
    job_title = 'Null'
    try:
        job_title = clean_text(soup.find('h1', class_='job-detail__info--title').get_text(separator=' '))
    except AttributeError:
        pass

    # Role
    role = 'Null'
    try:
        role = soup.find('div', class_='job-tags').find('a')['title']
        role = role.removeprefix("Chuyên môn ")
    except (AttributeError, TypeError):
        pass

    # Job description, location, time range, benefits
    location = time_range = benefits = 'Null'
    describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
    try:
        tag_div = soup.find('div', class_='job-description')
        items = tag_div.find_all('div', class_='job-description__item')
        for sub_items in items:
            name_tag = sub_items.find('h3').get_text()
            if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Quyền lợi': benefits = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Địa điểm làm việc': location = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())

        # Xử lý trường hợp không có mô tả hoặc yêu cầu
        if describe != 'Null' and required != 'Null':
            description = describe + '\n' + required
        elif describe != 'Null':
            description = describe
        elif required != 'Null':
            description = required
        else:
            description = 'Null'

    except AttributeError:
        description = 'Null'

    # Salary, province, experience years
    salary = province = experience_years = 'Null'
    try:
        tag_spe = soup.find('div', class_='job-detail__info--sections')
        list_spe = tag_spe.select('div.job-detail__info--section-content-value')
        if len(list_spe) >= 3:
            salary, province, experience_years = [tag.get_text(strip=True) for tag in list_spe[:3]]
    except AttributeError:
        pass  # All values remain 'Null'

    # Company name
    company_name = 'Null'
    try:
        company_name = soup.find('a', class_='name').get_text(strip=True)
    except AttributeError:
        pass

    # Company URL
    url_company = 'Null'
    try:
        tag_url = soup.find('div', class_='job-detail__company--link')
        url_company = tag_url.find('a')['href']
    except (AttributeError, TypeError):
        pass

    # Other job details: level, academic level, job type, recruitment_count
    level = academic_level = job_type = recruitment_count = 'Null'
    try:
        tag_lat = soup.select('div.box-general-group-info-value')
        if len(tag_lat) >= 4:
            level = tag_lat[0].get_text(strip=True)
            academic_level = tag_lat[1].get_text(strip=True)
            job_type = tag_lat[3].get_text(strip=True)
            recruitment_count = tag_lat[2].get_text(strip=True)
    except AttributeError:
        pass  # All values remain 'Null'

    # Skills
    skills = 'Null'
    try:
        skills = soup.find('div', class_='box-category collapsed').find('div', class_='box-category-tags').get_text(separator=', ', strip=True)
    except AttributeError:
        pass

    # Deadline
    deadline = 'Null'
    try:
        deadline = soup.find('div', class_='job-detail__info--deadline').get_text(strip=True)
        deadline = deadline.removeprefix("Hạn nộp hồ sơ: ")
    except AttributeError:
        pass
    

    # Thêm dòng vào DataFrame
    df.loc[len(df)] = [job_title, description, role, time_range, skills, location, province,
                        experience_years, salary, company_name, url_company, job_type,
                        deadline, academic_level, benefits, level, url, recruitment_count, date_crawl]


In [21]:
# scrape crawl_data_for_each_brand_url
def crawl_data_special_brand_url(html, url, date_crawl):

    soup = BeautifulSoup(html, "html.parser")
    if not soup:
        failed_urls.append(url)
    
    # Job title
    job_title = 'Null'
    try:
        job_title = clean_text(soup.find('h2', class_='premium-job-basic-information__content--title').get_text(separator=' '))
    except AttributeError:
        pass

    # Role
    role = 'Null'
    try:
        role = soup.find('div', class_='job-tags').find('a')['title']
        role = role.removeprefix("Chuyên môn ")
    except (AttributeError, TypeError):
        pass

    # Job description, time range, benefits, location
    time_range = benefits = location = description = 'Null'
    describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
    try:
        tag_div = soup.find('div', class_='premium-job-description') 
        items = tag_div.find_all('div', class_='premium-job-description__box', recursive=False) # chỉ lấy các div cùng cấp.
        for sub_items in items:
            name_tag = sub_items.find('h2', class_='premium-job-description__box--title').get_text()
            if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='premium-job-description__box--content').get_text())
            elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='premium-job-description__box--content').get_text())
            elif name_tag == 'Quyền lợi được hưởng': benefits = clean_text(sub_items.find('div', class_='premium-job-description__box--content').get_text())    
            elif name_tag == 'Địa điểm làm việc': location = clean_text(sub_items.find('div').get_text())          
            elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='premium-job-description__box--content').get_text())

        # Xử lý trường hợp không có mô tả hoặc yêu cầu
        if describe != 'Null' and required != 'Null':
            description = describe + '\n' + required
        elif describe != 'Null':
            description = describe
        elif required != 'Null':
            description = required
        else:
            description = 'Null'
    except AttributeError:
        pass

    # Salary, experience years, province
    salary = experience_years = province = 'Null'
    try:
        items = soup.find('div', class_='premium-job-basic-information__content--sections').find_all('div', class_='basic-information-item', recursive=False)
        for sub_items in items:
            name_tag = sub_items.find('div', class_='basic-information-item__data--label').get_text()
            if name_tag == 'Mức lương': salary = clean_text(sub_items.find('div', class_='basic-information-item__data--value').get_text())
            elif name_tag == 'Địa điểm': province = clean_text(sub_items.find('div', class_='basic-information-item__data--value').get_text())
            elif name_tag == 'Kinh nghiệm': experience_years = clean_text(sub_items.find('div', class_='basic-information-item__data--value').get_text())
    except AttributeError:
        pass  # All values remain 'Null'

    # level, academic_level, recruitment_count, job_type, deadline
    level = academic_level = recruitment_count = job_type = deadline = 'Null'
    try:
        items = soup.find('div', class_='premium-job-general-information__content').find_all('div', class_='premium-job-general-information__content--row', recursive=False)
        for sub_items in items:
            name_tag = sub_items.find('div', class_='general-information-data__label').get_text()
            if name_tag == 'Cấp bậc': level = clean_text(sub_items.find('div', class_='general-information-data__value').get_text())
            elif name_tag == 'Học vấn': academic_level = clean_text(sub_items.find('div', class_='general-information-data__value').get_text())
            elif name_tag == 'Số lượng tuyển': recruitment_count = clean_text(sub_items.find('div', class_='general-information-data__value').get_text())
            elif name_tag == 'Hình thức làm việc': job_type = clean_text(sub_items.find('div', class_='general-information-data__value').get_text())
            elif name_tag == 'Hạn nộp hồ sơ': deadline = clean_text(sub_items.find('div', class_='general-information-data__value').get_text())
    except AttributeError:
        pass  # All values remain 'Null'

    # Company name
    company_name = 'Null'
    try:
        company_name = soup.find('h1', class_='company-content__title--name').get_text(strip=True)
    except AttributeError:
        pass

    # Company URL
    url_company = 'Null'
    try:
        url_company = soup.find('div', class_='premium-job-header__company--tab-list').find('a')['href']
    except (AttributeError, TypeError):
        pass

    # Skills
    skills = 'Null'
    try:
        skills = soup.find('div', class_='premium-job-related-tags__section box-category collapsed').find('div', class_='premium-job-related-tags__section--tags').get_text(separator=', ', strip=True)
    except AttributeError:
        pass



    # Thêm dòng vào DataFrame
    df.loc[len(df)] = [job_title, description, role, time_range, skills, location, province,
                        experience_years, salary, company_name, url_company, job_type,
                        deadline, academic_level, benefits, level, url, recruitment_count, date_crawl]

In [26]:
# Hàm crawl dữ liệu cho từng BRAND_URL
def crawl_data_for_each_brand_url(html, url, date_crawl):

    soup = BeautifulSoup(html, "html.parser")
    if not soup:
        failed_urls.append(url)
    
    # Job title
    job_title = 'Null'
    try:
        job_title = clean_text(soup.find('h2', class_='title').get_text(separator=' '))
    except AttributeError:
        pass

    if job_title == 'Null':
        crawl_data_special_brand_url(html,url,date_crawl)
        return


    # Role
    role = 'Null'
    try:
        role = soup.find('div', class_='job-tags').find('a')['title']
        role = role.removeprefix("Chuyên môn ")
    except (AttributeError, TypeError):
        pass

    # Job description, time range, benefits, province
    time_range = benefits = province = 'Null'
    describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
    try:
        tag_div = soup.find('div', class_='box-job-info')
        try:
            province = tag_div.find('div', class_='box-address').find('div').get_text(strip=True) # Lưu ý lúc này province không phải chỉ chứa mỗi tỉnh => Cần xử lý ở giai đoạn sau.
        except AttributeError:
            pass
        items = tag_div.find_all('div', class_='box-info')[1:]
        for sub_items in items:
            name_tag = sub_items.find('h2', class_='title').get_text()
            if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='content-tab').get_text())
            elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='content-tab').get_text())
            elif name_tag == 'Quyền lợi được hưởng': benefits = clean_text(sub_items.find('div', class_='content-tab').get_text())               
            elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='content-tab').get_text())

        # Xử lý trường hợp không có mô tả hoặc yêu cầu
        if describe != 'Null' and required != 'Null':
            description = describe + '\n' + required
        elif describe != 'Null':
            description = describe
        elif required != 'Null':
            description = required
        else:
            description = 'Null'

    except AttributeError:
        description = 'Null'

    # Salary, experience years, level, academic level, job type
    salary = experience_years = level = academic_level = job_type = recruitment_count = 'Null'
    try:
        items = soup.find('div', class_='box-main').find_all('div', class_='box-item')
        for sub_items in items:
            name_tag = sub_items.find('strong').get_text()
            if name_tag == 'Mức lương': salary = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Hình thức làm việc ': job_type = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Cấp bậc ': level = clean_text(sub_items.find('span').get_text())           
            elif name_tag == 'Học vấn ': academic_level = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Kinh nghiệm ': experience_years = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Số lượng tuyển ': recruitment_count = clean_text(sub_items.find('span').get_text())
    except AttributeError:
        pass  # All values remain 'Null'

    # Company name, location
    company_name = location = 'Null'
    try:
        company_name = soup.find('div', class_='footer-info-content footer-info-company-name').get_text(strip=True)
    except AttributeError:
        pass

    try:
        location = soup.find('div',class_='footer-info-content footer-info-company-name').find_next_sibling('div', class_='footer-info-content').get_text(strip=True)
    except AttributeError:
        pass


    # Company URL
    url_company = 'Null'
    try:
        url_company = soup.find('a', class_='navbar-diamond-company-brand')['href']
    except (AttributeError, TypeError):
        pass

    # Skills
    skills = 'Null'
    try:
        skills = soup.find('div', class_='box-skill box-category collapsed').find('div', class_='item').get_text(separator=', ', strip=True)
    except AttributeError:
        pass

    # Deadline
    deadline = 'Null' # ở các brand_url không cung cấp deadline

    # Thêm dòng vào DataFrame
    df.loc[len(df)] = [job_title, description, role, time_range, skills, location, province,
                        experience_years, salary, company_name, url_company, job_type,
                        deadline, academic_level, benefits, level, url, recruitment_count, date_crawl]



# Read file json

In [37]:
data_0 = []

with open("_1-1000_bo_anti_cloudflaed.json", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        try:
            data_0.append(json.loads(line))  # Giải mã từng dòng
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc dòng: {line}")


In [38]:
data_1 = []

with open("_1000-2000_bo_anti_cloudflaed.json", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        try:
            data_1.append(json.loads(line))  # Giải mã từng dòng
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc dòng: {line}")


In [39]:
data_2 = []

with open("_2000-3000_bo_anti_cloudflaed.json", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        try:
            data_2.append(json.loads(line))  # Giải mã từng dòng
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc dòng: {line}")


In [98]:
data_3 = []

with open("_3000-4000_bo_anti_cloudflaed.json", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        try:
            data_3.append(json.loads(line))  # Giải mã từng dòng
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc dòng: {line}")

In [119]:
data_4 = []

with open("_4000-4235_bo_anti_cloudflaed.json", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        try:
            data_4.append(json.loads(line))  # Giải mã từng dòng
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc dòng: {line}")

---
# Scraping and decoding Date_Crawl_Module1 attribute

## 1-1000 recoreds

In [79]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job_Title", "Description", "Role", "Time_Range", "Skills", "Location_Detail", "Province", 
                               "Experience_Years", "Salary", "Company_Name", "Company_URL", "Job_Type", 
                               "Deadline", "Academic_Level", "Benefits", "Level", "URL", 'Recruitment_Count', 'Date_Crawl_Module1'])

failed_urls = []         

In [48]:
# Lặp qua từng trang đã crawl data_0
for idx, item in enumerate(tqdm(data_0, desc="Processing HTMLs")):
    html = item["html"]
    url = item["url"]
    # Kiểm tra kiểu dữ liệu của crawl_start_time
    crawl_start_time = item["crawl_start_time"]
    
    # Nếu crawl_start_time là một số, chuyển đổi nó thành chuỗi trước khi xử lý
    if isinstance(crawl_start_time, int):
        crawl_start_time = str(crawl_start_time)

    datetime_str = crawl_start_time.split(" ")[0]
    
    if '/brand/' in url:
        crawl_data_for_each_brand_url(html, url, datetime_str)
    else: crawl_data_for_each_normal_url(html, url, datetime_str)

Processing HTMLs: 100%|██████████| 1000/1000 [05:41<00:00,  2.93it/s]


In [61]:
df[df['Job_Title'] == 'Null']

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
755,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/nhan-vien-kiem-s...,Null,1746176847714
834,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/senior-angular-d...,Null,1746147101346



- từ 1-1000 records có 2 records không crawl được.
=> Xoá 2 record đó

In [ ]:
# Xoá cột toàn null các thuộc tính quan trọng
cols_to_check = df.columns.difference(['URL', 'Date_Crawl_Module1'])
mask = (df[cols_to_check] == 'Null').all(axis=1)
df = df[~mask]

In [67]:
print("các url không thể parses: ", len(failed_urls))
df.shape

các url không thể parses:  0


(998, 19)

In [68]:
df.head()

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Korean Bridge Software Engineer - Kỹ Sư Cầu...,Bridge in communication with Korean clients on...,Kỹ sư cầu nối BrSE,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),Null,"- Hà Nội: Tầng 15, Keangnam Landmark 72, Nam T...",Hà Nội,1 năm,Thoả thuận,Công Ty TNHH LG CNS VIỆT NAM,https://www.topcv.vn/cong-ty/cong-ty-tnhh-lg-c...,Toàn thời gian,16/05/2025,Đại Học trở lên,Attractive salary and bonus will be discussed ...,Nhân viên,https://www.topcv.vn/viec-lam/korean-bridge-so...,1 người,1746131931350
1,Network Technical Support Specialist,"As a Network Technical Support Specialist, you...",Technical Leader,Null,Null,"- Bắc Ninh: Số 126 Nguyễn Đăng Đạo, Phường Đại...",Bắc Ninh & 2 nơi khác,3 năm,Thoả thuận,CÔNG TY TNHH CÔNG NGHỆ ECOLIV,https://www.topcv.vn/cong-ty/cong-ty-tnhh-cong...,Toàn thời gian,17/05/2025,Cao Đẳng trở lên,1. A 13th month salary as an end-of-year bonus...,Nhân viên,https://www.topcv.vn/viec-lam/network-technica...,1 người,1746175303766
2,Gaming SOP Designer,Thiết kế thumbnail YouTube cho kênh gaming (De...,Game Developer,Null,Null,"- Đà Nẵng: 49 Hóa Sơn, Hải Châu, Hải Châu\n- H...","Đà Nẵng, Hà Nội",1 năm,10 - 13 triệu,CÔNG TY TNHH MTV NIA-STUDIO,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,22/05/2025,Trung học phổ thông (Cấp 3) trở lên,Mức lương: thỏa thuận.\nLương tháng thứ 13\nCá...,Nhân viên,https://www.topcv.vn/viec-lam/gaming-sop-desig...,1 người,1746175335597
3,Lập Trình Viên Backend Java -Thu Nhập Từ 16...,- Tham gia vào các công đoạn: kiến trúc hệ thố...,Backend Developer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30),"Redis, kafka, RESTful APIs, Kiểm Thử Phần Mềm ...","- Hà Nội: Tòa nhà MB 18 Lê Văn Lương, Thanh Xu...",Hà Nội,3 năm,16 - 32 triệu,Công ty Cổ phần SIMBATECH,https://www.topcv.vn/cong-ty/cong-ty-co-phan-s...,Toàn thời gian,30/06/2025,Đại Học trở lên,CHẾ ĐỘ ĐÃI NGỘ:- Mức lương từ 16 – 32 triệu tù...,Nhân viên,https://www.topcv.vn/viec-lam/lap-trinh-vien-b...,5 người,1746132001925
4,Chuyên Viên Kiểm Thử Phần Mềm / Tester - Thu...,"Thiết kế và thực hiện kế hoạch kiểm thử, các t...",Manual Tester,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30),"phân tích yêu cầu, Viết Testcase, Thực Hiện Te...","- Hà Nội: Tòa nhà MB 18 Lê Văn Lương, Thanh Xu...",Hà Nội,3 năm,16 - 32 triệu,Công ty Cổ phần SIMBATECH,https://www.topcv.vn/cong-ty/cong-ty-co-phan-s...,Toàn thời gian,30/06/2025,Đại Học trở lên,"Quyền lợi:Thu nhập: Mức lương từ 16,000,000 VN...",Nhân viên,https://www.topcv.vn/viec-lam/chuyen-vien-kiem...,5 người,1746132020149


In [74]:
# Bước 1: Đảm bảo cột là số
df_temp_0 = df.copy()
df_temp_0['Date_Crawl_Module1'] = df_temp_0['Date_Crawl_Module1'].astype(float)

# Bước 2: Chuyển timestamp (ms) thành datetime và định dạng chuỗi đầy đủ
df_temp_0['Date_Crawl_Module1'] = pd.to_datetime(df_temp_0['Date_Crawl_Module1'], unit='ms') \
                                .dt.strftime('%d/%m/%Y %H:%M:%S')

In [77]:
df_temp_0.head(3)

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Korean Bridge Software Engineer - Kỹ Sư Cầu...,Bridge in communication with Korean clients on...,Kỹ sư cầu nối BrSE,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),Null,"- Hà Nội: Tầng 15, Keangnam Landmark 72, Nam T...",Hà Nội,1 năm,Thoả thuận,Công Ty TNHH LG CNS VIỆT NAM,https://www.topcv.vn/cong-ty/cong-ty-tnhh-lg-c...,Toàn thời gian,16/05/2025,Đại Học trở lên,Attractive salary and bonus will be discussed ...,Nhân viên,https://www.topcv.vn/viec-lam/korean-bridge-so...,1 người,01/05/2025 20:38:51
1,Network Technical Support Specialist,"As a Network Technical Support Specialist, you...",Technical Leader,Null,Null,"- Bắc Ninh: Số 126 Nguyễn Đăng Đạo, Phường Đại...",Bắc Ninh & 2 nơi khác,3 năm,Thoả thuận,CÔNG TY TNHH CÔNG NGHỆ ECOLIV,https://www.topcv.vn/cong-ty/cong-ty-tnhh-cong...,Toàn thời gian,17/05/2025,Cao Đẳng trở lên,1. A 13th month salary as an end-of-year bonus...,Nhân viên,https://www.topcv.vn/viec-lam/network-technica...,1 người,02/05/2025 08:41:43
2,Gaming SOP Designer,Thiết kế thumbnail YouTube cho kênh gaming (De...,Game Developer,Null,Null,"- Đà Nẵng: 49 Hóa Sơn, Hải Châu, Hải Châu\n- H...","Đà Nẵng, Hà Nội",1 năm,10 - 13 triệu,CÔNG TY TNHH MTV NIA-STUDIO,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,22/05/2025,Trung học phổ thông (Cấp 3) trở lên,Mức lương: thỏa thuận.\nLương tháng thứ 13\nCá...,Nhân viên,https://www.topcv.vn/viec-lam/gaming-sop-desig...,1 người,02/05/2025 08:42:15


In [85]:
df_temp_0.shape

(998, 19)

---

## 1000-2000 records

In [80]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job_Title", "Description", "Role", "Time_Range", "Skills", "Location_Detail", "Province", 
                               "Experience_Years", "Salary", "Company_Name", "Company_URL", "Job_Type", 
                               "Deadline", "Academic_Level", "Benefits", "Level", "URL", 'Recruitment_Count', 'Date_Crawl_Module1'])
                
failed_urls = []

In [81]:
# Lặp qua từng trang đã crawl data_1
for idx, item in enumerate(tqdm(data_1, desc="Processing HTMLs")):
    html = item["html"]
    url = item["url"]
    # Kiểm tra kiểu dữ liệu của crawl_start_time
    crawl_start_time = item["crawl_start_time"]
    
    # Nếu crawl_start_time là một số, chuyển đổi nó thành chuỗi trước khi xử lý
    if isinstance(crawl_start_time, int):
        crawl_start_time = str(crawl_start_time)

    datetime_str = crawl_start_time.split(" ")[0]
    
    if '/brand/' in url:
        crawl_data_for_each_brand_url(html, url, datetime_str)
    else: crawl_data_for_each_normal_url(html, url, datetime_str)

Processing HTMLs: 100%|██████████| 1000/1000 [05:31<00:00,  3.02it/s]


In [82]:
df[df['Job_Title'] == 'Null']

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1



- Không có records nào không cào được

In [83]:
print("các url không thể parses: ", len(failed_urls))
df.shape

các url không thể parses:  0


(1000, 19)

In [84]:
df.head()

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Chuyên Viên Tester - OS Bank,"1. Xây dựng kịch bản test cho ứng dụng: web, m...",Software Tester (Automation & Manual),Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"phân tích yêu cầu, Kiểm thử phần mềm, Viết Tes...","- Hà Nội: 18 Lê Văn Lương, Cầu Giấy",Hà Nội,3 năm,15 - 35 triệu,Công Ty Cổ Phần GoGroup,https://www.topcv.vn/cong-ty/cong-ty-co-phan-g...,Toàn thời gian,24/05/2025,Đại Học trở lên,"- Làm việc từ thứ 2 đến thứ 6, nghỉ thứ 7, CN-...",Nhân viên,https://www.topcv.vn/viec-lam/chuyen-vien-test...,2 người,1746185451852
1,Fullstack Java,INFINIQ is a pioneering South Korean corporati...,Software Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Java, Javascript, Python, ReactJS","- Hà Nội: Lot 2C1 Mac Thai Tong Street, Trung ...",Hà Nội,2 năm,Thoả thuận,"INFINIQ Vietnam Co.,ltd",https://www.topcv.vn/cong-ty/infiniq-vietnam-c...,Toàn thời gian,31/05/2025,Đại Học trở lên,Top 3 reasons to join us:Attractive salaryInte...,Nhân viên,https://www.topcv.vn/viec-lam/fullstack-java/1...,3 người,1746185476406
2,Database Administrator,"Quản trị, cài đặt, tối ưu hóa và duy trì cơ sở...",Database Administrator (DBA),Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"SQL, Thiết Kế Cơ Sở Dữ Liệu, Quản Trị Cơ Sở Dữ...","- Hà Nội: Tòa 319 Bộ Quốc Phòng, 63 Lê Văn Lươ...",Hà Nội,3 năm,25 - 40 triệu,Công ty Cổ phần Hệ thống Công nghệ ETC,https://www.topcv.vn/cong-ty/cong-ty-co-phan-h...,Toàn thời gian,31/05/2025,Đại Học trở lên,"Chế độ lương thưởng hấp dẫn,Chế độ BHYT, BHXH,...",Nhân viên,https://www.topcv.vn/viec-lam/database-adminis...,5 người,1746185498476
3,Server Engineer,"Participate in the requirements analysis, solu...",System Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30)\n8h00 - 17h...,"Linux Unix System Administration, Networking (...","- Hà Nội: Leadvisor Tower, 643 Phạm Văn Đồng, ...",Hà Nội,Không yêu cầu,Thoả thuận,Công ty TNHH ZTE HK (Việt Nam),https://www.topcv.vn/cong-ty/cong-ty-tnhh-zte-...,Toàn thời gian,24/05/2025,Đại Học trở lên,Attractive salary and compensation benefitsGai...,Nhân viên,https://www.topcv.vn/viec-lam/server-engineer/...,1 người,1746185524263
4,KỸ SƯ CẦU NỐI BrSE,MÔ TẢ DỰ ÁN:- Tham gia vào các dự án outso...,Kỹ sư cầu nối BrSE,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Bridge Engineer, Software Engineering, Quản Lý...",- Hồ Chí Minh: Tân Bình,Hồ Chí Minh,2 năm,Thoả thuận,GMO-Z.com RUNSYSTEM,https://www.topcv.vn/cong-ty/gmo-z-com-runsyst...,Toàn thời gian,31/05/2025,Đại Học trở lên,Lương: UPTO $1800 GROSS (theo vị trí và năng ...,Nhân viên,https://www.topcv.vn/viec-lam/ky-su-cau-noi-br...,2 người,1746185546895


In [86]:
# Bước 1: Đảm bảo cột là số
df_temp_1 = df.copy()
df_temp_1['Date_Crawl_Module1'] = df_temp_1['Date_Crawl_Module1'].astype(float)

# Bước 2: Chuyển timestamp (ms) thành datetime và định dạng chuỗi đầy đủ
df_temp_1['Date_Crawl_Module1'] = pd.to_datetime(df_temp_1['Date_Crawl_Module1'], unit='ms') \
                                .dt.strftime('%d/%m/%Y %H:%M:%S')

In [87]:
df_temp_1.head(3)

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Chuyên Viên Tester - OS Bank,"1. Xây dựng kịch bản test cho ứng dụng: web, m...",Software Tester (Automation & Manual),Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"phân tích yêu cầu, Kiểm thử phần mềm, Viết Tes...","- Hà Nội: 18 Lê Văn Lương, Cầu Giấy",Hà Nội,3 năm,15 - 35 triệu,Công Ty Cổ Phần GoGroup,https://www.topcv.vn/cong-ty/cong-ty-co-phan-g...,Toàn thời gian,24/05/2025,Đại Học trở lên,"- Làm việc từ thứ 2 đến thứ 6, nghỉ thứ 7, CN-...",Nhân viên,https://www.topcv.vn/viec-lam/chuyen-vien-test...,2 người,02/05/2025 11:30:51
1,Fullstack Java,INFINIQ is a pioneering South Korean corporati...,Software Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Java, Javascript, Python, ReactJS","- Hà Nội: Lot 2C1 Mac Thai Tong Street, Trung ...",Hà Nội,2 năm,Thoả thuận,"INFINIQ Vietnam Co.,ltd",https://www.topcv.vn/cong-ty/infiniq-vietnam-c...,Toàn thời gian,31/05/2025,Đại Học trở lên,Top 3 reasons to join us:Attractive salaryInte...,Nhân viên,https://www.topcv.vn/viec-lam/fullstack-java/1...,3 người,02/05/2025 11:31:16
2,Database Administrator,"Quản trị, cài đặt, tối ưu hóa và duy trì cơ sở...",Database Administrator (DBA),Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"SQL, Thiết Kế Cơ Sở Dữ Liệu, Quản Trị Cơ Sở Dữ...","- Hà Nội: Tòa 319 Bộ Quốc Phòng, 63 Lê Văn Lươ...",Hà Nội,3 năm,25 - 40 triệu,Công ty Cổ phần Hệ thống Công nghệ ETC,https://www.topcv.vn/cong-ty/cong-ty-co-phan-h...,Toàn thời gian,31/05/2025,Đại Học trở lên,"Chế độ lương thưởng hấp dẫn,Chế độ BHYT, BHXH,...",Nhân viên,https://www.topcv.vn/viec-lam/database-adminis...,5 người,02/05/2025 11:31:38


In [88]:
df_temp_1.shape

(1000, 19)

---
## 2000-3000 records

In [89]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job_Title", "Description", "Role", "Time_Range", "Skills", "Location_Detail", "Province", 
                               "Experience_Years", "Salary", "Company_Name", "Company_URL", "Job_Type", 
                               "Deadline", "Academic_Level", "Benefits", "Level", "URL", 'Recruitment_Count', 'Date_Crawl_Module1'])
                
failed_urls = []

In [90]:
# Lặp qua từng trang đã crawl data_2
for idx, item in enumerate(tqdm(data_2, desc="Processing HTMLs")):
    html = item["html"]
    url = item["url"]
    # Kiểm tra kiểu dữ liệu của crawl_start_time
    crawl_start_time = item["crawl_start_time"]
    
    # Nếu crawl_start_time là một số, chuyển đổi nó thành chuỗi trước khi xử lý
    if isinstance(crawl_start_time, int):
        crawl_start_time = str(crawl_start_time)

    datetime_str = crawl_start_time.split(" ")[0]
    
    if '/brand/' in url:
        crawl_data_for_each_brand_url(html, url, datetime_str)
    else: crawl_data_for_each_normal_url(html, url, datetime_str)

Processing HTMLs: 100%|██████████| 1000/1000 [05:53<00:00,  2.83it/s]


In [91]:
df[df['Job_Title'] == 'Null']

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
219,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/chuyen-vien-cong...,Null,1746213956487
463,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/technical-suppor...,Null,1746218865807
990,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/user-experience-...,Null,1746256119934
991,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/ai-engineer/1635...,Null,1746229848042


\- Có 4 records không scrape được các thuộc tính chính => Xoá

In [92]:
# Xoá cột toàn null các thuộc tính quan trọng
cols_to_check = df.columns.difference(['URL', 'Date_Crawl_Module1'])
mask = (df[cols_to_check] == 'Null').all(axis=1)
df = df[~mask]

In [93]:
print("các url không thể parses: ", len(failed_urls))
df.shape

các url không thể parses:  0


(996, 19)

In [94]:
df.head()

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,IT Infrastructure And Operations,Network Infrastructure OperationEnsure the net...,IT Helpdesk/IT support,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Quản Lý Hệ Thống, An Ninh Mạng, Kiến Thức Về M...","- Bà Rịa-Vũng Tàu: Lô1, Khu công nghiệp Phú Mỹ...",Bà Rịa-Vũng Tàu,Không yêu cầu,Thoả thuận,CÔNG TY TNHH POSCO DX VIỆT NAM,https://www.topcv.vn/cong-ty/cong-ty-tnhh-posc...,Toàn thời gian,17/05/2025,Đại Học trở lên,"· From 14,000,000 net · 13th-month salary· ...",Nhân viên,https://www.topcv.vn/viec-lam/it-infrastructur...,1 người,1746209504526
1,Lập Trình Viên Phần Mềm,- Phát triển và duy trì ứng dụng di động đa nề...,Fullstack Developer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00)\nThứ 7 (từ ...,"Giải quyết vấn đề, Kiểm Thử Phần Mềm (testing)...","- Hồ Chí Minh: 46 Nguyễn Thị Định, P.An Phú, Q...",Hồ Chí Minh,2 năm,Thoả thuận,CHI NHÁNH CÔNG TY TNHH ECHO MEDI,https://www.topcv.vn/cong-ty/chi-nhanh-cong-ty...,Toàn thời gian,17/05/2025,Cao Đẳng trở lên,"- Mức lương cạnh tranh, điều chỉnh theo năng l...",Nhân viên,https://www.topcv.vn/viec-lam/lap-trinh-vien-p...,1 người,1746209525381
2,Nhân Viên It Help Desk,"−Cài đặt máy tính, phần mềm, các thiết bị ngoạ...",IT Helpdesk/IT support,Null,"Khắc Phục Sự Cố Máy Tính, Hỗ Trợ Người Dùng, K...","- Hà Nội: Lô 4, KCN Quang Minh, Mê Linh",Hà Nội,Dưới 1 năm,8 - 12 triệu,Công ty TNHH MTV Chuyển phát nhanh Thuận Phong...,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,30/06/2025,Cao Đẳng trở lên,"Lương: 8 - 12trPhúc lợi:Tham gia BHXH, BHYT, B...",Nhân viên,https://www.topcv.vn/viec-lam/nhan-vien-it-hel...,2 người,1746209544830
3,Senior Game Designer,Thiết kế gameplay (lối chơi) bao gồm game mech...,Game Design,Null,Null,"- Hà Nội: Tầng 11, tòa nhà diamondflower, 48 L...",Hà Nội,5 năm,Thoả thuận,CÔNG TY CỔ PHẦN PHÁT TRIỂN CÔNG NGHỆ ARCHER ST...,https://www.topcv.vn/cong-ty/cong-ty-co-phan-p...,Toàn thời gian,29/05/2025,Đại Học trở lên,Làm việc cùng đội ngũ nhân sự nhiều năm kinh n...,Nhân viên,https://www.topcv.vn/viec-lam/senior-game-desi...,1 người,1746209568334
4,Kỹ Sư Cầu Nối Brse (JLPT N2),"Trao đổi, thương lượng và thu thập yêu cầu dự ...",Kỹ sư cầu nối BrSE,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Kiến Thức Chuyên Sâu Về Cầu Nối Brse, Kỹ Năng ...",- Hà Nội: FLC Landmark Tower Ngõ 60 Dương Khuê...,Hà Nội,2 năm,Thoả thuận,Công ty TNHH Lisod Việt Nam,https://www.topcv.vn/cong-ty/cong-ty-tnhh-liso...,Toàn thời gian,31/08/2025,Đại Học trở lên,Mức lương khởi điểm hấp dẫn tuỳ thuộc năng lực...,Nhân viên,https://www.topcv.vn/viec-lam/ky-su-cau-noi-br...,2 người,1746209593520


In [95]:
# Bước 1: Đảm bảo cột là số
df_temp_2 = df.copy()
df_temp_2['Date_Crawl_Module1'] = df_temp_2['Date_Crawl_Module1'].astype(float)

# Bước 2: Chuyển timestamp (ms) thành datetime và định dạng chuỗi đầy đủ
df_temp_2['Date_Crawl_Module1'] = pd.to_datetime(df_temp_2['Date_Crawl_Module1'], unit='ms') \
                                .dt.strftime('%d/%m/%Y %H:%M:%S')

In [96]:
df_temp_2.head(3)

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,IT Infrastructure And Operations,Network Infrastructure OperationEnsure the net...,IT Helpdesk/IT support,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Quản Lý Hệ Thống, An Ninh Mạng, Kiến Thức Về M...","- Bà Rịa-Vũng Tàu: Lô1, Khu công nghiệp Phú Mỹ...",Bà Rịa-Vũng Tàu,Không yêu cầu,Thoả thuận,CÔNG TY TNHH POSCO DX VIỆT NAM,https://www.topcv.vn/cong-ty/cong-ty-tnhh-posc...,Toàn thời gian,17/05/2025,Đại Học trở lên,"· From 14,000,000 net · 13th-month salary· ...",Nhân viên,https://www.topcv.vn/viec-lam/it-infrastructur...,1 người,02/05/2025 18:11:44
1,Lập Trình Viên Phần Mềm,- Phát triển và duy trì ứng dụng di động đa nề...,Fullstack Developer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00)\nThứ 7 (từ ...,"Giải quyết vấn đề, Kiểm Thử Phần Mềm (testing)...","- Hồ Chí Minh: 46 Nguyễn Thị Định, P.An Phú, Q...",Hồ Chí Minh,2 năm,Thoả thuận,CHI NHÁNH CÔNG TY TNHH ECHO MEDI,https://www.topcv.vn/cong-ty/chi-nhanh-cong-ty...,Toàn thời gian,17/05/2025,Cao Đẳng trở lên,"- Mức lương cạnh tranh, điều chỉnh theo năng l...",Nhân viên,https://www.topcv.vn/viec-lam/lap-trinh-vien-p...,1 người,02/05/2025 18:12:05
2,Nhân Viên It Help Desk,"−Cài đặt máy tính, phần mềm, các thiết bị ngoạ...",IT Helpdesk/IT support,Null,"Khắc Phục Sự Cố Máy Tính, Hỗ Trợ Người Dùng, K...","- Hà Nội: Lô 4, KCN Quang Minh, Mê Linh",Hà Nội,Dưới 1 năm,8 - 12 triệu,Công ty TNHH MTV Chuyển phát nhanh Thuận Phong...,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,30/06/2025,Cao Đẳng trở lên,"Lương: 8 - 12trPhúc lợi:Tham gia BHXH, BHYT, B...",Nhân viên,https://www.topcv.vn/viec-lam/nhan-vien-it-hel...,2 người,02/05/2025 18:12:24


In [97]:
df_temp_2.shape

(996, 19)

---
## 3000-4000 records

In [135]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job_Title", "Description", "Role", "Time_Range", "Skills", "Location_Detail", "Province", 
                               "Experience_Years", "Salary", "Company_Name", "Company_URL", "Job_Type", 
                               "Deadline", "Academic_Level", "Benefits", "Level", "URL", 'Recruitment_Count', 'Date_Crawl_Module1'])
                
failed_urls = []

In [136]:
# Lặp qua từng trang đã crawl data_3
for idx, item in enumerate(tqdm(data_3, desc="Processing HTMLs")):
    html = item["html"]
    url = item["url"]
    crawl_start_time = item["crawl_start_time"]
    
    # Nếu crawl_start_time là None, gán nó thành 'Null'
    if crawl_start_time is None:
        crawl_start_time = 'Null'

    # Nếu crawl_start_time là số, chuyển thành chuỗi và chuyển đổi thành datetime
    if isinstance(crawl_start_time, (int, float)):
        crawl_start_time = pd.to_datetime(crawl_start_time, unit='ms').strftime('%d/%m/%Y %H:%M:%S')

    # Kiểm tra nếu html là chuỗi hợp lệ trước khi tiếp tục
    if isinstance(html, str) and html.strip():  # Kiểm tra nếu html là chuỗi không rỗng
        if '/brand/' in url:
            crawl_data_for_each_brand_url(html, url, datetime_str)
        else:
            crawl_data_for_each_normal_url(html, url, datetime_str)
    else:
        failed_urls.append(url)  # Thêm URL vào danh sách failed_urls nếu không có HTML hợp lệ

Processing HTMLs: 100%|██████████| 1000/1000 [06:03<00:00,  2.75it/s]


In [137]:
df[df['Job_Title'] == 'Null']

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
99,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/thuc-tap-sinh-ne...,Null,Null
253,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/unity-game-devel...,Null,Null
286,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/thuc-tap-sinh-bu...,Null,Null
287,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/thuc-tap-sinh-ba...,Null,Null
288,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/thuc-tap-sinh-fr...,Null,Null
305,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/brand/nganhangthuongmaico...,Null,Null
322,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/mobile-developer...,Null,Null
323,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/senior-back-end-...,Null,Null
324,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/tester-leader/12...,Null,Null
327,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/player-support-s...,Null,Null


In [138]:
a = len(df[df['Date_Crawl_Module1'] == 'Null'])  # Đếm số dòng có giá trị 'Null'
print(a)


996


\- Có 996/1000 records không chứa dữ liệu date_crawl

In [139]:
# Xoá cột toàn null các thuộc tính quan trọng
cols_to_check = df.columns.difference(['URL', 'Date_Crawl_Module1'])
mask = (df[cols_to_check] == 'Null').all(axis=1)
df = df[~mask]

In [140]:
print("các url không thể parses: ", len(failed_urls))
df.shape

các url không thể parses:  4


(975, 19)

In [141]:
for i in failed_urls:
    print(i)

https://www.topcv.vn/viec-lam/lap-trinh-vien-unity-mobile-game/1145456.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/ky-su-speech-processing-engineer/1686879.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/automation-tester/1686880.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/ios-developer/1686873.html?ta_source=ITJobs_LinkDetail


In [142]:
df_temp_3 = df.copy()

In [143]:
df_temp_3.head(3)

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Middle DevOps Lĩnh Vực Trung Gian Thanh Toán,Chịu trách nhiệm vận hành hệ thống on-prem với...,DevOps Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00)\nThời gian ...,"Kiến Thức Về Hệ Điều Hành Linux, Tự Động Hóa B...","- Hà Nội: Toà nhà Hà Nội Ford, 311 Trường Chin...",Hà Nội,2 năm,20 - 30 triệu,CÔNG TY CỔ PHẦN THƯƠNG MẠI ĐIỆN TỬ BẢO KIM,https://www.topcv.vn/cong-ty/cong-ty-co-phan-t...,Toàn thời gian,10/05/2025,Đại Học trở lên,Môi trường làm việc trẻ trung năng độngReview ...,Nhân viên,https://www.topcv.vn/viec-lam/middle-devops-li...,1 người,Null
1,Senior AI Engineer,"Thiết kế, huấn luyện và triển khai các Large L...",AI Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),"Phát Triển Mô Hình Học Máy (machine Learning),...","- Hồ Chí Minh: 25/1 Cửu Long Phường 2, Tân Bình",Hồ Chí Minh,2 năm,Thoả thuận,CÔNG TY TNHH AI POWER,https://www.topcv.vn/cong-ty/cong-ty-tnhh-ai-p...,Toàn thời gian,31/05/2025,Đại Học trở lên,"Làm việc trong dự án AI thực chiến, quy mô lớn...",Nhân viên,https://www.topcv.vn/viec-lam/senior-ai-engine...,1 người,Null
2,Android Developer,Develop and optimize Android applications usin...,Software Engineer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30),"Android SDK, Android studio, Kiến Trúc Phần Mề...","- Đà Nẵng: 343 Nguyễn Hữu Thọ, Cẩm Lệ",Đà Nẵng,Dưới 1 năm,8 - 15 triệu,CÔNG TY TNHH MTV LIMGROW,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,10/05/2025,Đại Học trở lên,"Working hours: Monday to Friday, 8:00 AM - 5:3...",Nhân viên,https://www.topcv.vn/viec-lam/android-develope...,1 người,Null


In [144]:
df_temp_3.shape

(975, 19)

---
## 4000-4235 records

In [121]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job_Title", "Description", "Role", "Time_Range", "Skills", "Location_Detail", "Province", 
                               "Experience_Years", "Salary", "Company_Name", "Company_URL", "Job_Type", 
                               "Deadline", "Academic_Level", "Benefits", "Level", "URL", 'Recruitment_Count', 'Date_Crawl_Module1'])
                
failed_urls = []

In [122]:
# Lặp qua từng trang đã crawl data_4
for idx, item in enumerate(tqdm(data_4, desc="Processing HTMLs")):
    html = item["html"]
    url = item["url"]
    crawl_start_time = item["crawl_start_time"]
    
    # Nếu crawl_start_time là None, gán nó thành 'Null'
    if crawl_start_time is None:
        crawl_start_time = 'Null'

    # Nếu crawl_start_time là số, chuyển thành chuỗi và chuyển đổi thành datetime
    if isinstance(crawl_start_time, (int, float)):
        crawl_start_time = pd.to_datetime(crawl_start_time, unit='ms').strftime('%d/%m/%Y %H:%M:%S')

    # Kiểm tra nếu html là chuỗi hợp lệ trước khi tiếp tục
    if isinstance(html, str) and html.strip():  # Kiểm tra nếu html là chuỗi không rỗng
        if '/brand/' in url:
            crawl_data_for_each_brand_url(html, url, datetime_str)
        else:
            crawl_data_for_each_normal_url(html, url, datetime_str)
    else:
        failed_urls.append(url)  # Thêm URL vào danh sách failed_urls nếu không có HTML hợp lệ

Processing HTMLs: 100%|██████████| 235/235 [01:21<00:00,  2.89it/s]


In [123]:
df[df['Job_Title'] == 'Null']

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
168,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/php-web-develope...,Null,Null
172,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/lap-trinh-web-ph...,Null,Null
176,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/lap-trinh-vien-b...,Null,Null
177,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/senior-developer...,Null,Null
212,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/senior-business-...,Null,Null
221,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,https://www.topcv.vn/viec-lam/it-communicator-...,Null,Null


In [127]:
a = len(df[df['Date_Crawl_Module1'] == 'Null'])  # Đếm số dòng có giá trị 'Null'
print(a)


235


\- Tất cả dữ liệu không có date crawl module 1

In [128]:
# Xoá cột toàn null các thuộc tính quan trọng
cols_to_check = df.columns.difference(['URL', 'Date_Crawl_Module1'])
mask = (df[cols_to_check] == 'Null').all(axis=1)
df = df[~mask]

In [129]:
print("các url không thể parses: ", len(failed_urls))
df.shape

các url không thể parses:  0


(229, 19)

In [131]:
df_temp_4 = df.copy()

In [133]:
df_temp_4.shape

(229, 19)

---
# Check and delete duplicate records

In [145]:
# Gộp các DataFrame lại
df_combined = pd.concat([df_temp_0, df_temp_1, df_temp_2, df_temp_3, df_temp_4], ignore_index=True)

# Bỏ qua cột 'Date_Crawl_Module1' và kiểm tra các dòng bị trùng lặp theo các cột còn lại
df_without_date = df_combined.drop(columns=['Date_Crawl_Module1'])

# Kiểm tra các dòng trùng lặp
duplicates = df_without_date[df_without_date.duplicated()]

# Số lượng dòng bị duplicate
duplicate_count = duplicates.shape[0]

# In ra số lượng dòng bị duplicate
print(f"Số lượng dòng bị duplicate (ngoại trừ 'Date_Crawl_Module1'): {duplicate_count}")

Số lượng dòng bị duplicate (ngoại trừ 'Date_Crawl_Module1'): 12


In [146]:
df_combined.shape

(4198, 19)

In [147]:

# Xóa các dòng duplicate và chỉ giữ lại một dòng duy nhất
df_combined_no_duplicates = df_combined.drop_duplicates(subset=df_without_date.columns, keep='first')

# In kết quả sau khi loại bỏ duplicate
print(f"Số dòng sau khi loại bỏ duplicate: {df_combined_no_duplicates.shape[0]}")

Số dòng sau khi loại bỏ duplicate: 4186


In [148]:
df_combined_no_duplicates.head()

,Job_Title,Description,Role,Time_Range,Skills,Location_Detail,Province,Experience_Years,Salary,Company_Name,Company_URL,Job_Type,Deadline,Academic_Level,Benefits,Level,URL,Recruitment_Count,Date_Crawl_Module1
0,Korean Bridge Software Engineer - Kỹ Sư Cầu...,Bridge in communication with Korean clients on...,Kỹ sư cầu nối BrSE,Thứ 2 - Thứ 6 (từ 08:00 đến 17:00),Null,"- Hà Nội: Tầng 15, Keangnam Landmark 72, Nam T...",Hà Nội,1 năm,Thoả thuận,Công Ty TNHH LG CNS VIỆT NAM,https://www.topcv.vn/cong-ty/cong-ty-tnhh-lg-c...,Toàn thời gian,16/05/2025,Đại Học trở lên,Attractive salary and bonus will be discussed ...,Nhân viên,https://www.topcv.vn/viec-lam/korean-bridge-so...,1 người,01/05/2025 20:38:51
1,Network Technical Support Specialist,"As a Network Technical Support Specialist, you...",Technical Leader,Null,Null,"- Bắc Ninh: Số 126 Nguyễn Đăng Đạo, Phường Đại...",Bắc Ninh & 2 nơi khác,3 năm,Thoả thuận,CÔNG TY TNHH CÔNG NGHỆ ECOLIV,https://www.topcv.vn/cong-ty/cong-ty-tnhh-cong...,Toàn thời gian,17/05/2025,Cao Đẳng trở lên,1. A 13th month salary as an end-of-year bonus...,Nhân viên,https://www.topcv.vn/viec-lam/network-technica...,1 người,02/05/2025 08:41:43
2,Gaming SOP Designer,Thiết kế thumbnail YouTube cho kênh gaming (De...,Game Developer,Null,Null,"- Đà Nẵng: 49 Hóa Sơn, Hải Châu, Hải Châu\n- H...","Đà Nẵng, Hà Nội",1 năm,10 - 13 triệu,CÔNG TY TNHH MTV NIA-STUDIO,https://www.topcv.vn/cong-ty/cong-ty-tnhh-mtv-...,Toàn thời gian,22/05/2025,Trung học phổ thông (Cấp 3) trở lên,Mức lương: thỏa thuận.\nLương tháng thứ 13\nCá...,Nhân viên,https://www.topcv.vn/viec-lam/gaming-sop-desig...,1 người,02/05/2025 08:42:15
3,Lập Trình Viên Backend Java -Thu Nhập Từ 16...,- Tham gia vào các công đoạn: kiến trúc hệ thố...,Backend Developer,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30),"Redis, kafka, RESTful APIs, Kiểm Thử Phần Mềm ...","- Hà Nội: Tòa nhà MB 18 Lê Văn Lương, Thanh Xu...",Hà Nội,3 năm,16 - 32 triệu,Công ty Cổ phần SIMBATECH,https://www.topcv.vn/cong-ty/cong-ty-co-phan-s...,Toàn thời gian,30/06/2025,Đại Học trở lên,CHẾ ĐỘ ĐÃI NGỘ:- Mức lương từ 16 – 32 triệu tù...,Nhân viên,https://www.topcv.vn/viec-lam/lap-trinh-vien-b...,5 người,01/05/2025 20:40:01
4,Chuyên Viên Kiểm Thử Phần Mềm / Tester - Thu...,"Thiết kế và thực hiện kế hoạch kiểm thử, các t...",Manual Tester,Thứ 2 - Thứ 6 (từ 08:00 đến 17:30),"phân tích yêu cầu, Viết Testcase, Thực Hiện Te...","- Hà Nội: Tòa nhà MB 18 Lê Văn Lương, Thanh Xu...",Hà Nội,3 năm,16 - 32 triệu,Công ty Cổ phần SIMBATECH,https://www.topcv.vn/cong-ty/cong-ty-co-phan-s...,Toàn thời gian,30/06/2025,Đại Học trở lên,"Quyền lợi:Thu nhập: Mức lương từ 16,000,000 VN...",Nhân viên,https://www.topcv.vn/viec-lam/chuyen-vien-kiem...,5 người,01/05/2025 20:40:20


---
# Export to json and csv

In [149]:
df.to_csv('4186_records_data_bronze.csv', index=False)

In [150]:
df.to_json('4186_records_data_bronze.json', orient='records', force_ascii=False)

---